# Transonic shock buffet on a swept wing

In [ ]:
from os.path import join
from os import makedirs
from collections import defaultdict
from copy import deepcopy
from math import sqrt
import torch as pt
import numpy as np
from pandas import read_csv
from scipy.signal import welch
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from flowtorch.analysis import SVD, OptDMD, HOOptDMD
from pydmd import BOPDMD
from notebook_utils import set_seed

data_path = "data"
results_path = join("results", "swept_wing")
makedirs(results_path, exist_ok=True)
plt.rcParams["figure.dpi"] = 160
plt.rcParams["text.usetex"] = True
pt.set_default_dtype(pt.float64)

## Preliminary processing and analysis

In [ ]:
def crop_mask(mask: pt.Tensor, left: int=20, right: int=10, up: int=15, down: int=15) -> pt.Tensor:
    mask[:left, :] = 0
    mask[-right:, :] = 0
    mask[:, :up] = 0
    mask[:, -down:] = 0
    return mask

In [ ]:
cp_raw = pt.load(join(data_path, "wing_ipsp_ma084_alpha4_cp.pt"))
mask = crop_mask(pt.load(join(data_path, "wing_ipsp_ma084_alpha4_mask.pt")))

In [ ]:
cp_raw.shape

In [ ]:
data = {
    "cp": cp_raw*mask.unsqueeze(-1),
    "mask": mask,
    "x": pt.load(join(data_path, "wing_ipsp_ma084_alpha4_x.pt")),
    "y": pt.load(join(data_path, "wing_ipsp_ma084_alpha4_y.pt"))
}
data.keys()

In [ ]:
dt = 1.0 / 2000.5
CHORD = 0.1965
U_INF = 215.0

In [ ]:
fb = 0.4*U_INF/CHORD
1.0/fb/dt, 1000*dt*fb, fb

In [ ]:
dm = data["cp"].clamp(-1.5, 0.5).flatten(0, 1)[:, 500:1500].type(pt.float64)
dm_mean = dm.mean(dim=1)
dm -= dm_mean.unsqueeze(-1)

In [ ]:
cp_std = dm.std(dim=1).reshape(data["x"].shape)
x_norm = (data["x"] - data["x"].min()) / (data["x"].max() - data["x"].min())
y_norm = (data["y"] - data["y"].min()) / (data["y"].max() - data["y"].min())

In [ ]:
frame = pt.zeros_like(x_norm)
frame[0, :] = 1.0
frame[-1:,:] = 1.0
frame[:, 0] = 1.0
frame[:, -1:] = 1.0

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(5, 2.5), sharex=True, sharey=True)

dm_norm = (dm_mean - dm_mean.min()) / (dm_mean.max() - dm_mean.min())
dm_std = dm.std(dim=1)
std_norm = (dm_std - dm_std.min()) / (dm_std.max() - dm_std.min())
ax1.contourf(x_norm, y_norm, dm_norm.reshape(x_norm.shape), cmap="jet", levels=120)
cont = ax2.contourf(x_norm, y_norm, std_norm.reshape(x_norm.shape), cmap="jet", levels=120)
ax1.contour(x_norm, y_norm, frame, levels=[0.99], cmap="binary", vmin=0.0, vmax=1.0, linewidths=1.0)
ax2.contour(x_norm, y_norm, frame, levels=[0.99], cmap="binary", vmin=0.0, vmax=1.0, linewidths=1.0)
x_probe = x_norm[335:340, 55:60].mean().item()
y_probe = y_norm[335:340, 55:60].mean().item()
ax2.scatter([x_probe], [y_probe], marker="o", s=20, facecolor="none", edgecolor="C2", lw=2)
ax2.annotate("probe", xy=(x_probe, y_probe), xytext=(0.1, y_probe), arrowprops=dict(arrowstyle="->", color="C2"), xycoords="axes fraction", zorder=8)
plt.colorbar(cont, ax=(ax1, ax2))
fig.supxlabel(r"$\tilde{x}$", y=-0.05)
fig.supylabel(r"$\tilde{y}$")
ax1.set_title(r"$\mathrm{mean}(c_p)$")
ax2.set_title(r"$\mathrm{std}(c_p)$")
plt.savefig(join(results_path, "cp_mean_std.png"), bbox_inches="tight")

In [ ]:
dyn_data = read_csv(join(data_path, "wing_sensors_ma084_alpha4.csv"))
dyn_data.head()

In [ ]:
def extract_ipsp_window(ipsp_busy: np.ndarray, time: np.ndarray) -> tuple:
    active = ipsp_busy > 1.0
    t_start = time[active][0]
    return t_start, active
t_start, include = extract_ipsp_window(dyn_data["IPSP_Busy"].values, dyn_data["Time"].values)

In [ ]:
t = dyn_data["Time"].values[include] - t_start
# probe at (1020.515, -203.283)
p0 = dyn_data["KUP10402"].values[include]
# probe at (1061.890, -367.473)
p1 = dyn_data["KUP10803"].values[include]
# probe at (1125.556, -500.388)
p2 = dyn_data["KUP11803"].values[include]
# acceleration wing tip
acc = dyn_data["Acc_Stbd"].values[include]

In [ ]:
ipsp_probe = data["cp"][335:340, 55:60, :].mean(dim=(0, 1)).numpy()

In [ ]:
dtp = t[1] - t[0]
n_samples = len(t)
f0, a0 = welch(p0-p0.mean(), 1.0/dtp, nperseg=int(n_samples/5), nfft=n_samples*2)
f1, a1 = welch(p1-p1.mean(), 1.0/dtp, nperseg=int(n_samples/5), nfft=n_samples*2)
f2, a2 = welch(p2-p2.mean(), 1.0/dtp, nperseg=int(n_samples/5), nfft=n_samples*2)
fac, aac = welch(acc-acc.mean(), 1.0/dtp, nperseg=int(n_samples/5), nfft=n_samples*2)
f_ipsp, a_ipsp = welch(ipsp_probe-ipsp_probe.mean(), 2000.5, nperseg=int(len(ipsp_probe)/5), nfft=2*len(ipsp_probe))

In [ ]:
fig, ax = plt.subplots(figsize=(6, 2))
#ax.plot(f0*CHORD/U_INF, a0/a0.max(), ls=":", lw=1, label=r"$p_0$")
ax.plot(f1*CHORD/U_INF, a1/a1.max(), ls="-", lw=1, label=r"$p_1$")
#ax.plot(f2*CHORD/U_INF, a2/a2.max(), ls="-.", lw=1, label=r"$p_2$")
ax.plot(fac*CHORD/U_INF, aac/aac.max(), ls="--", lw=1, label=r"$a_z$")
ax.plot(f_ipsp*CHORD/U_INF, a_ipsp/a_ipsp.max(), ls="-", lw=1, label=r"$IPSP$")
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlim(1e-2, 1.0)
ax.set_ylim(1e-3)
ax.set_xlabel(r"$S_r$")
ax.set_ylabel("PSD")
ax.legend(ncol=4)
ax.grid(ls="--")
plt.show()

## DMD analysis

In [ ]:
svd = SVD(dm)
print(svd)

In [ ]:
dm_pro = svd.U.T @ dm

### Comparison of exact and optimized DMD

Only the first 750 snapshots are used to fit the operator. The reconstruction loss is computed on all snapshots.

In [ ]:
set_seed(0)
optDMD = HOOptDMD(dm, dt, rank_dr=svd.opt_rank, rank=svd.opt_rank)
optDMD.train(batch_size=32, stopping_options={"checkpoint" : "/tmp/best_model.pt", "patience" : 100})

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
e = range(len(optDMD.log["train_loss"]))
ax.plot(e, optDMD.log["train_loss"], c=f"C0", ls="-", label="training")
ax.plot(e, optDMD.log["val_loss"], c=f"C1", ls="--", label="validation")
ax.set_xlabel(r"$e$")
ax.set_ylabel(r"$L_2$")
ax.legend(ncol=2)
e_min_loss = pt.tensor(optDMD.log["val_loss"]).argmin()
ax.axvline(e_min_loss, ls=":", c="k")
ax.text(e_min_loss-4, 1.15, "checkpoint", rotation=90)
ax.set_xlim(0, list(e)[-1])
plt.show()

In [ ]:
times = pt.arange(0.0, 1000*dt, dt)

In [ ]:
bdmd = BOPDMD(svd_rank=svd.opt_rank, num_trials=0, eig_constraints={"conjugate_pairs"},
                 varpro_opts_dict={"verbose": True, "use_fulljac" : True})
bdmd.fit(dm[:, :750].numpy(), times[:750].numpy())

In [ ]:
dmr = svd.U @ pt.diag(svd.s) @ svd.V.T

In [ ]:
eigvals = pt.exp(pt.from_numpy(bdmd.eigs)*dt)
rec = pt.from_numpy(bdmd.modes) @ (pt.linalg.vander(eigvals, N=dm.size(1)) * pt.from_numpy(bdmd.amplitudes).unsqueeze(-1))
berr = (dmr - rec).norm() / dmr.norm()

In [ ]:
rec = optDMD.modes @ (pt.linalg.vander(optDMD.eigvals, N=dm.size(1)) * optDMD.amplitude.unsqueeze(-1))
err = (dmr - rec).real.norm() / dmr.norm()

In [ ]:
berr, err

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

ax.add_patch(plt.Circle((0.0, 0.0), radius=1.0, color="k", alpha=0.2, ec="k", lw=2, ls="--"))
ax.scatter(optDMD.dmd_init.eigvals.real, optDMD.dmd_init.eigvals.imag, marker="x", c="C0", label="exact DMD")
bev = pt.exp(pt.from_numpy(bdmd.eigs)*dt)
ax.scatter(optDMD.eigvals.real, optDMD.eigvals.imag, marker="+", s=50, c="C2", label="ADAM, POD", zorder=7)
ax.scatter(bev.real, bev.imag, marker="*", c="C3", label="VarPro")
ax.set_aspect("equal")
ax.set_ylim(0)
ax.legend()
ax.set_xlabel(r"$\Re(\lambda)$")
ax.set_ylabel(r"$\Im(\lambda)$")
plt.savefig(join(results_path, "eigval_comparison.pdf"), bbox_inches="tight")

### Optimized DMD with sparsity promotion

In [ ]:
sparsity = pt.arange(0.0, 0.105, 0.005)

In [ ]:
results = defaultdict(list)
set_seed(0)
for sp in sparsity:
    def loss_func(labels, predictions, eigvecs, eigvals):
        l2_loss = (labels - predictions).norm() / sqrt(labels.numel())
        sparsity_loss = eigvecs.norm(dim=0).mean()
        return l2_loss + sp*sparsity_loss
    dmd = OptDMD(dm_pro, dt, rank=svd.opt_rank)
    dmd.train(batch_size=32, loss_function=loss_func, stopping_options={"checkpoint" : "/tmp/best_model.pt", "patience" : 100})
    dmd.load_state_dict(pt.load("/tmp/best_model.pt"))
    results["err"].append(dmd.reconstruction_error.norm() / dm_pro[:, :-1].norm())
    results["card"].append((dmd.amplitude > 1e-2).sum().item())

In [ ]:
pt.save(results, join(results_path, "sparsity_influence.pt"))

In [ ]:
results = pt.load(join(results_path, "sparsity_influence.pt"))

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
ax_err = ax.twinx()
ax.plot(sparsity, results["card"], marker="x", c="k")
ax.set_xlim(0, 0.1)
err = pt.tensor(results["err"])
err_norm = (err / err.min() - 1) * 100
ax_err.plot(sparsity, err_norm, marker="+", c="C3")
ax.set_xlabel(r"$\gamma_0$")
ax.set_ylabel(r"$\mathrm{card}(\mathbf{b})$")
ax.set_ylim(0, 70)
ax_err.set_ylabel(r"$\Pi_{\mathrm{loss}}$ in $\%$")
ax_err.spines["right"].set_color("C3")
ax_err.tick_params(axis="y", colors="C3")
ax_err.yaxis.label.set_color("C3")
plt.savefig(join(results_path, "sparsity_effect.pdf"), bbox_inches="tight")

In [ ]:
set_seed(0)
def loss_func(labels, predictions, eigvecs, eigvals):
    l2_loss = (labels - predictions).norm() / sqrt(labels.numel())
    sparsity_loss = eigvecs.norm(dim=0).mean()
    return l2_loss + 0.03*sparsity_loss
dmd = OptDMD(dm_pro, dt, rank=svd.opt_rank)
dmd.train(batch_size=32, loss_function=loss_func, stopping_options={"checkpoint" : "/tmp/best_model.pt", "patience" : 120})
dmd.load_state_dict(pt.load("/tmp/best_model.pt"))

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
e = range(len(dmd.log["train_loss"]))
ax.plot(e, dmd.log["train_loss"], c=f"C0", ls="-", label="training")
ax.plot(e, dmd.log["val_loss"], c=f"C1", ls="--", label="validation")
ax.set_xlabel(r"$e$")
ax.set_ylabel(r"$L_2$")
ax.legend(ncol=2)
e_min_loss = pt.tensor(dmd.log["val_loss"]).argmin()
ax.axvline(e_min_loss, ls=":", c="k")
ax.text(e_min_loss-12, 1.3, "checkpoint", rotation=90)
ax.set_xlim(0, list(e)[-1])
plt.show()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, figsize=(6, 4), gridspec_kw={'height_ratios': [1, 1, 3]}, sharex=True)

ax1.plot(f_ipsp*CHORD/U_INF, a_ipsp/a_ipsp.max(), ls="-", lw=1, label=r"$c_p$ probe")
ax1.set_yscale("log")
ax1.set_ylim(8e-2)
ax1.grid(ls="--")
ax1.set_ylabel("PSD, $c_p$")
ax2.plot(fac*CHORD/U_INF, aac/aac.max(), ls="-", lw=1, label=r"$a_z$")
ax2.set_yscale("log")
ax2.set_ylim(1e-4)
ax2.set_ylabel("PSD, $a_z$")
ax2.grid(ls="--")

f = dmd.frequency * CHORD/U_INF
pos = f > 0.01
imp = dmd.integral_contribution
impr = imp[pos] / imp[pos].sum() * 100
markerline, stemlines, baseline = ax3.stem(f[pos].numpy(), impr.numpy(), basefmt="none", linefmt=f"k-", markerfmt="x")
plt.setp(stemlines, 'linewidth', 1)
plt.setp(markerline, markersize=6)
keep = impr > 1.0
sorting = f[pos][keep].sort().indices
for i, ind in enumerate(sorting):
    f_i, i_i = f[pos][keep][ind], impr[keep][ind]
    ax3.text(f_i+0.005, i_i*1.01, r"${:d}$".format(i), c="C3")
ax3.set_xlim(0, 0.6)
ax3.set_xlabel(r"$S_r$")
ax3.set_ylabel(r"$I_{\mathrm{rel}}$ in $\%$")
ax3.grid(ls="--")
ax3.set_ylim(0, 50)
ax3.set_xticks(pt.arange(0.0, 0.601, 0.05))
plt.savefig(join(results_path, "spectrum.pdf"), bbox_inches="tight")

In [ ]:
top_modes = dmd.modes[:, pos][:, keep][:, sorting]
top_imp = imp[pos][keep][sorting]
top_imp_rel = top_imp / top_imp.sum()
top_f = f[pos][keep][sorting]

In [ ]:
def add_mode(ax, mode, bounds = None, cmap="seismic", scale=1.0):
    if bounds is None:
        vmin, vmax = mode.min(), mode.max()
        vref = max(vmin.abs(), vmax.abs())*scale
        vmin, vmax = -vref, vref
    else:
        vmin, vmax = bounds
    levels = pt.linspace(vmin, vmax, 120)
    ax.contourf(x_norm, y_norm, mode, cmap=cmap, vmin=vmin, vmax=vmax, levels=levels)
    ax.contour(x_norm, y_norm, frame, levels=[0.99], cmap="binary", vmin=0.0, vmax=1.0, linewidths=1.0)

In [ ]:
fig, axarr = plt.subplots(len(top_f), 4, figsize=(6, 2*len(top_f)), sharex=True, sharey=True)

for i, (f_i, i_i) in enumerate(zip(top_f, top_imp_rel)):
    mode = (svd.U.type(top_modes.dtype) @ top_modes[:, i]).reshape(data["x"].shape)
    add_mode(axarr[i, 0], mode.real)
    add_mode(axarr[i, 1], mode.imag)
    add_mode(axarr[i, 2], mode.abs())
    angle = mode.angle()
    angle_masked = pt.where(cp_std > 0.09, angle, pt.tensor(0))
    add_mode(axarr[i, 3], angle_masked, (-np.pi, np.pi))
    axarr[i, 0].set_ylabel(r"$S_r={:2.2f}$, $I={:2.2f}$".format(f_i.item(), i_i.item()*100))
for ax in axarr.flatten():
    ax.set_xticks([])
    ax.set_yticks([])
plt.show()

In [ ]:
vis = (4, 1)

In [ ]:
fig, axarr = plt.subplots(2, 4, figsize=(6, 4), sharex=True, sharey=True)

for i, ii in enumerate(vis):
    mode = (svd.U.type(top_modes.dtype) @ top_modes[:, ii]).reshape(x_norm.shape)
    add_mode(axarr[i, 0], mode.real)
    add_mode(axarr[i, 1], mode.imag)
    add_mode(axarr[i, 2], mode.abs())
    angle = mode.angle()
    angle_masked = pt.where(cp_std > 0.07, angle, pt.tensor(0))
    add_mode(axarr[i, 3], angle_masked, (-np.pi, np.pi))
    axarr[i, -1].text(1.1, 0.5, r"$S_r={:2.2f}$, $I_{:s}={:2.2f}\%$".format(top_f[ii].item(), "\mathrm{rel}", top_imp_rel[ii].item()*100), rotation=-90, va="center")
axarr[0, 0].invert_yaxis()
axarr[0, 0].set_title(r"$\Re (\mathbf{\varphi})$")
axarr[0, 1].set_title(r"$\Im (\mathbf{\varphi})$")
axarr[0, 2].set_title(r"$|\mathbf{\varphi}|$")
axarr[0, 3].set_title(r"$\mathrm{tan}^{-1}( \Im (\mathbf{\varphi})/ \Re (\mathbf{\varphi}) )$")
fig.supxlabel(r"$\tilde{x}$")
fig.supylabel(r"$\tilde{y}$")
axarr[1, 2].annotate("", xy=(0.55, 0.76), xytext=(0.88, 0.65), arrowprops=dict(arrowstyle="<->"), xycoords="axes fraction", zorder=8)
axarr[1, 2].annotate("", xy=(0.45, 0.2), xytext=(0.55, 0.5), arrowprops=dict(arrowstyle="->"), xycoords="axes fraction", zorder=8)
axarr[0, 2].annotate("", xy=(0.88, 0.85), xytext=(0.72, 0.65), arrowprops=dict(arrowstyle="->"), xycoords="axes fraction", zorder=8)
plt.savefig(join(results_path, "modes.png"), bbox_inches="tight")

## Propagation velocity

In [ ]:
np_line_1 = 20
line_1s = pt.stack((
    pt.linspace(0.64, 0.47, np_line_1),
    pt.linspace(0.4, 0.85, np_line_1),
)).T
np_line_2 = 20
line_2s = pt.stack((
    pt.linspace(0.9, 0.68, np_line_2),
    pt.linspace(0.05, 0.35, np_line_2),
)).T

In [ ]:
fig, axarr = plt.subplots(1, 2, figsize=(3, 2), sharex=True, sharey=True)

lines = [line_1s, line_2s]
for i, ii in enumerate(vis):
    mode = (svd.U.type(top_modes.dtype) @ top_modes[:, ii]).reshape(x_norm.shape)
    add_mode(axarr[i], mode.abs())
    line = lines[i]
    axarr[i].plot(line[:, 0], line[:, 1], c="k", ls=":")
plt.show()

In [ ]:
angle_1 = (svd.U.type(top_modes.dtype) @ top_modes[:, vis[0]]).angle()
angle_2 = (svd.U.type(top_modes.dtype) @ top_modes[:, vis[1]]).angle()

In [ ]:
angle_l1 = []
for pi in range(line_1s.shape[0]):
    closest = (pt.stack((x_norm.flatten(), y_norm.flatten()), dim=1) - line_1s[pi]).norm(dim=1).argmin()
    angle_l1.append(angle_1[closest])
angle_l1 = pt.stack(angle_l1)
angle_l2 = []
for pi in range(line_2s.shape[0]):
    closest = (pt.stack((x_norm.flatten(), y_norm.flatten()), dim=1) - line_2s[pi]).norm(dim=1).argmin()
    angle_l2.append(angle_2[closest])
angle_l2 = pt.stack(angle_l2)

In [ ]:
line_1 = pt.zeros_like(line_1s)
line_1[:, 0] = line_1s[:, 0] * (data["x"].max() - data["x"].min()) + data["x"].min()
line_1[:, 1] = line_1s[:, 1] * (data["y"].max() - data["y"].min()) + data["y"].min()
line_1 /= 1000
line_2 = pt.zeros_like(line_2s)
line_2[:, 0] = line_2s[:, 0] * (data["x"].max() - data["x"].min()) + data["x"].min()
line_2[:, 1] = line_2s[:, 1] * (data["y"].max() - data["y"].min()) + data["y"].min()
line_2 /= 1000

In [ ]:
fig, axarr = plt.subplots(1, 2, figsize=(6, 2), sharey=True, sharex=True)

# line 1 - inboard waves
ds_1 = (line_1 - line_1[0, :]).norm(dim=1)
axarr[0].scatter(ds_1, angle_l1, marker="x", c="k", s=10)
low_1, up_1 = 0.1, 0.25
l1_range = pt.logical_and(ds_1 > low_1, ds_1 < up_1)
dmp = pt.cat((angle_l1[l1_range].unsqueeze(-1), line_1[l1_range, 0].unsqueeze(-1)), dim=1).T
dmp_mean = dmp.mean(dim=1)
U, s, VH = pt.linalg.svd(dmp - dmp_mean.unsqueeze(-1), full_matrices=False)
dphi_ds_1 = U[0, 0] / U[1, 0]
u_rel_1 = 2*np.pi*top_f[vis[0]]*U_INF/CHORD / dphi_ds_1 / U_INF
phi_1_rec = dmp_mean[0] + dphi_ds_1 * (line_1[l1_range, 0] - dmp_mean[1])
axarr[0].plot(ds_1[l1_range], phi_1_rec, ls="--", c="C0", label="linear fit")
axarr[0].set_title(r"$U_p=" + f"{u_rel_1:2.4f}" + "U_\infty$")

# line 2 - pressure side towards trailing edge
ds_2 = (line_2 - line_2[0, :]).norm(dim=1)
axarr[1].scatter(ds_2, angle_l2, marker="x", c="k", s=10)
low_2, up_2 = 0.0, 0.12
l2_range = pt.logical_and(ds_2 > low_2, ds_2 < up_2)
dmp = pt.cat((angle_l2[l2_range].unsqueeze(-1), line_2[l2_range, 0].unsqueeze(-1)), dim=1).T
dmp_mean = dmp.mean(dim=1)
U, s, VH = pt.linalg.svd(dmp - dmp_mean.unsqueeze(-1), full_matrices=False)
dphi_ds_2 = U[0, 0] / U[1, 0]
u_rel_2 = 2*np.pi*top_f[vis[1]]*U_INF/CHORD / dphi_ds_2 / U_INF
phi_2_rec = dmp_mean[0] + dphi_ds_2 * (line_2[l2_range, 0] - dmp_mean[1])
axarr[1].plot(ds_2[l2_range], phi_2_rec, ls="--", c="C0", label="linear fit")
axarr[1].set_title(r"$U_p=" + f"{u_rel_2:2.4f}" + "U_\infty$")
plt.show()

In [ ]:
wave_length_1 = pt.abs(u_rel_1*U_INF/(top_f[vis[0]]*U_INF/CHORD))
wave_length_2 = pt.abs(u_rel_2*U_INF/(top_f[vis[1]]*U_INF/CHORD))
wave_length_1.item(), wave_length_2.item()

In [ ]:
wave_length_1.item() / CHORD, wave_length_2.item() / CHORD

## Animation of selected modes

In [ ]:
def animate_reconstruction(reconstruction, title, fps, n_frames):
    fig, ax = plt.subplots(1, figsize=(2.5, 3))
    #plt.subplots_adjust(bottom=0.0, top=1.0, left=0.05, right=0.95, wspace=-0.1)
    vref = max(reconstruction.min().abs(), reconstruction.max().abs())
    levels = pt.linspace(-vref, vref, 120)
    conts = []
    conts.append(ax.contourf(x_norm, y_norm, reconstruction[:, 0].reshape(x_norm.shape), levels=levels, vmin=-vref, vmax=vref, cmap="seismic"))
    ax.contour(x_norm, y_norm, frame, levels=[0.99], cmap="binary", vmin=0.0, vmax=1.0, linewidths=1.0)
    ax.set_axis_off()
    ax.set_title(title)
    for c in conts[0].collections:
        c.set_edgecolor("face")
    def update(frame):
        print(f"\rProcessing frame {frame}", end="")
        for c in conts[0].collections:
            c.remove()
        conts[0] = ax.contourf(x_norm, y_norm, reconstruction[:, frame].reshape(x_norm.shape), levels=levels, vmin=-vref, vmax=vref, cmap="seismic")
        return conts
    return FuncAnimation(fig, update, frames=n_frames, blit=False, interval=fps)

In [ ]:
top_k = dmd.top_modes(10, integral=True, f_min=0.005)

In [ ]:
sr = dmd.frequency * CHORD / U_INF
im = dmd.integral_contribution / top_imp.sum() * 100
dmd_ani = deepcopy(dmd)
dmd_ani._eigvals = pt.nn.Parameter(dmd._eigvals / dmd._eigvals.abs())
for i in top_k:
    rec = dmd_ani.partial_reconstruction({i})
    rec = svd.U @ rec
    title = r"$S_r={:2.2f}$, $I_{:s}={:2.2f}\%$".format(dmd.frequency[i]*CHORD/U_INF, "{rel}", im[i])
    anim = animate_reconstruction(rec, title, 10, 50)
    anim.save(join(results_path, f"mode_{i}.mp4"), fps=10, dpi=320, bitrate=1500)
    plt.close()